In [1]:
pip install transformers-interpret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 794.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/

In [ ]:
from transformers_interpret import TokenClassificationExplainer
from transformers import AutoModelForTokenClassification, AutoTokenizer

model_path = "xlmr-ner-amharic-ecommerce"

model = AutoModelForTokenClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

explainer = TokenClassificationExplainer(
    model=model,
    tokenizer=tokenizer,
    labels=model.config.id2label
)

# Example tokenized message to interpret
text = "ዋጋ፦ 1400 ብር አድራሻ መገናኛ ቢሮ ቁ. S05"

# Interpret it
word_attributions = explainer(text)

# Print results
for word, attribution in word_attributions:
    print(f"{word:10} --> {attribution:.4f}")


In [ ]:
pip install lime scikit-learn

In [ ]:
from lime.lime_text import LimeTextExplainer
import numpy as np

class_names = list(model.config.id2label.values())

# Define a wrapper for your model
def predict_proba(texts):
    encoded = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, is_split_into_words=False)
    outputs = model(**encoded)
    logits = outputs.logits
    probs = torch.nn.functional.softmax(logits, dim=-1)
    # Return probs for first token (you can adjust to your use case)
    return probs[:, 1].detach().cpu().numpy()  # Dummy example

explainer = LimeTextExplainer(class_names=class_names)

exp = explainer.explain_instance(
    "ቤት ውስጥ የሚጠቀም በኤሌክትሪክ መቀየሪያ እና 1400 ብር ዋጋ ነው",
    predict_proba,
    num_features=10
)

exp.show_in_notebook()
